# Lab 02

Представлен набор данных о городских квартирах. Рассчитайте следующие величины:

* Оценку отношения R параметров 'Полная площадь квартиры' (full_sq) и 'Количество комнат' (num_room);
* стандартную ошибку данной оценки;
* долю p квартир, у которых 'Материал стен' = 4;
* левую и правые доверительные границы нормальной аппроксимации p, доверительная вероятность = 95.3%.

Объем генеральной совокупности равен 41468.    

*Примечание*. Под дисперсией и стандартным отклонением подразумеваются их выборочные (несмещенные) аналоги.
    
Итоговый файл **.csv** должен содержать 5 чисел, через запятую.

In [21]:
from math import sqrt
from pathlib import Path
from typing import List, Any

import numpy as np

from scipy.stats import norm
import pandas as pd

In [22]:
N = 41468
alpha = 1 - (95.3/100)

file_path = Path('.')
data_path = file_path / '..' / 'data' / 'lw2' / 'projects' / '1032173841' / 'data.csv'
data_path = data_path.resolve()

data = pd.read_csv(data_path)
n = len(data)
f = n/N

display(data)

,full_sq,life_sq,material,num_room
0,90,61,5,4
1,51,24,1,1
2,39,20,1,1
3,63,58,1,2
4,76,45,1,3
...,...,...,...,...
6131,52,30,1,2
6132,43,29,1,2
6133,29,18,2,1
6134,37,37,1,1


In [23]:
def mean(lst: List[float]) -> float:
    return sum(lst)/len(lst)

R = mean(data.full_sq) / mean(data.num_room)
print("R", R)

R 27.520752159918686


In [24]:
def std_err(X, Y, R):
    err = (
            sqrt(1-f)     * sqrt(sum([(y_i - R*x_i)**2 for x_i, y_i in zip(X, Y)])) /
#       ---------------------------------------------------------------------------
        (sqrt(n) * mean(X) * sqrt(n-1))
    )
    return err
std = std_err(data.num_room, data.full_sq, R)
print("std_err", std)

std_err 0.10787992025156828


In [25]:
def median(lst: List[float]) -> float:
    sort = sorted(lst)
    mid = (len(lst) - 1) // 2
    
    if (len(lst) % 2) == 0:
        return float(sort[mid])
    else:
        return float((sort[mid] + sort[mid + 1])/2)

In [26]:
def var(lst: List[float]) -> float:
    var = (
        sum([(l_i - mean(lst))**2 for l_i in lst]) / 
    #   ------------------------------------------
                         (len(lst)-1)
    )
    return var

In [27]:
a = len([m for m in data.material if m == 4])
p = a/n
q = 1 - p
print("a", a)
print("p", p)

a 1058
p 0.17242503259452413


In [28]:
t = norm.ppf(1-(alpha/2))

di_l = p - (t*sqrt(1-f)*sqrt(p*q/(n-1)) + 1/(2*n))
di_r = p + (t*sqrt(1-f)*sqrt(p*q/(n-1)) + 1/(2*n))

print(di_l, di_r)

0.16596423539740301 0.17888582979164525


In [29]:
data = {
    'R':     [R],
    's_R':   [std],
    'p':     [p],
    'left':  [di_l],
    'right': [di_r],
}

df = pd.DataFrame(data=data)
display(df)

csv_path = Path('.') / 'lab02.csv'
csv_path = csv_path.resolve()

if csv_path.exists():
    csv_path.unlink()

df.to_csv(csv_path, index=False)

,R,s_R,p,left,right
0,27.520752,0.10788,0.172425,0.165964,0.178886
